In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load  Data

In [2]:
mnist_full = pd.read_csv('./data/train.csv')
mnist_test = pd.read_csv('./data/test.csv')
mnist_full

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
mnist_full['label'].value_counts()

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

In [4]:
y = mnist_full['label'].to_numpy()
y

array([1, 0, 1, ..., 7, 6, 9])

In [5]:
X = mnist_full.drop(columns='label').to_numpy().reshape(-1, 28, 28, 1)
X.shape

(42000, 28, 28, 1)

### Split Data

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [8]:
X_train.shape

(31500, 28, 28, 1)

## Build Model

In [9]:
import tensorflow as tf
from tensorflow.keras import *

In [10]:
model = Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(64, (4,4), activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

Metal device set to: Apple M1


2022-03-13 17:31:06.175775: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-13 17:31:06.176630: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 64)        1088      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 12, 12, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 128)         131200    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 128)        0         
 2D)                                                             
                                                        

In [12]:
model.compile(optimizer=optimizers.SGD(learning_rate=0.09), loss='sparse_categorical_crossentropy', metrics=['acc'])

In [13]:
early_stop = callbacks.EarlyStopping(monitor='val_acc', patience=5, restore_best_weights=True)

In [15]:
model.fit(X_train, y_train, 
          epochs=50, 
          batch_size=100, 
          validation_data=[X_val, y_val], 
          validation_batch_size=100, 
          callbacks=[early_stop],
          verbose=1)

2022-03-13 17:35:42.415219: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-13 17:35:42.567256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50
313/315 [============================>.] - ETA: 0s - loss: 0.1808 - acc: 0.9417

2022-03-13 17:35:48.866779: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


315/315 [==============================] - 7s 22ms/step - loss: 0.1801 - acc: 0.9418 - val_loss: 0.0662 - val_acc: 0.9793
Epoch 2/50
315/315 [==============================] - 7s 22ms/step - loss: 0.0573 - acc: 0.9810 - val_loss: 0.0456 - val_acc: 0.9853
Epoch 3/50
315/315 [==============================] - 7s 21ms/step - loss: 0.0374 - acc: 0.9883 - val_loss: 0.0432 - val_acc: 0.9862
Epoch 4/50
315/315 [==============================] - 7s 22ms/step - loss: 0.0288 - acc: 0.9906 - val_loss: 0.0362 - val_acc: 0.9891
Epoch 5/50
315/315 [==============================] - 7s 22ms/step - loss: 0.0202 - acc: 0.9931 - val_loss: 0.0449 - val_acc: 0.9870
Epoch 6/50
315/315 [==============================] - 7s 22ms/step - loss: 0.0174 - acc: 0.9947 - val_loss: 0.0428 - val_acc: 0.9871
Epoch 7/50
315/315 [==============================] - 7s 22ms/step - loss: 0.0131 - acc: 0.9958 - val_loss: 0.0397 - val_acc: 0.9883
Epoch 8/50
315/315 [==============================] - 7s 22ms/step - loss: 0.010

## Retrain on entire data

In [22]:
model.fit(X, y, epochs=5, batch_size=100)

Epoch 1/5
420/420 [==============================] - 8s 19ms/step - loss: 0.0223 - acc: 0.9958
Epoch 2/5
420/420 [==============================] - 8s 19ms/step - loss: 0.0102 - acc: 0.9975
Epoch 3/5
420/420 [==============================] - 8s 20ms/step - loss: 0.0077 - acc: 0.9980
Epoch 4/5
420/420 [==============================] - 8s 20ms/step - loss: 0.0041 - acc: 0.9987
Epoch 5/5
420/420 [==============================] - 8s 20ms/step - loss: 0.0038 - acc: 0.9988


### Predict on Test data

In [16]:
mnist_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_test = mnist_test.to_numpy().reshape(-1, 28, 28, 1)
X_test.shape

(28000, 28, 28, 1)

In [18]:
y_pred = model.predict(X_test)
y_pred

2022-03-13 17:39:04.068632: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


array([[6.1374537e-11, 1.4226868e-12, 1.0000000e+00, ..., 5.2509511e-12,
        1.1059741e-10, 1.3962088e-12],
       [9.9999976e-01, 4.6362658e-10, 1.4120689e-09, ..., 8.5413693e-10,
        3.3718453e-10, 2.4610836e-08],
       [1.2339341e-12, 8.2699196e-13, 5.8346345e-13, ..., 4.2175725e-12,
        1.2550708e-08, 1.0000000e+00],
       ...,
       [6.2500135e-21, 2.4398814e-17, 3.5268768e-20, ..., 2.4696084e-17,
        3.1094979e-16, 3.4408467e-17],
       [8.9648688e-13, 9.2278477e-12, 8.1832483e-14, ..., 2.4908850e-10,
        2.3047089e-11, 9.9999988e-01],
       [4.4633221e-11, 1.7905767e-12, 1.0000000e+00, ..., 4.9635079e-12,
        4.6219459e-09, 2.2217493e-12]], dtype=float32)

In [19]:
predictions = y_pred.argmax(axis=-1)
predictions

array([2, 0, 9, ..., 3, 9, 2])

## Create Submission

In [20]:
submission = pd.DataFrame({'ImageId': range(1, len(y_pred) + 1), 'Label': predictions})
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [21]:
submission.to_csv('data/submission.csv', index=False)

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False